In [1]:
from SeeQ import *

# CQ Specification

In [2]:
# Here we specify our Computationa Quantities: The physical abstract quantities that we need to calculate as part of an application.

AHU_Tma = CQ(description="Mixed Air Temperature in the AHU",
             unit=UNIT.DEG_C,
             implementation=[
                    GraphCQ(0, [BRICK.AHU, BRICK.hasPoint, BRICK.Mixed_Air_Temperature_Sensor]),
                    GraphCQ(1, [BRICK.AHU, BRICK.hasPart, BRICK.Mixed_Damper, BRICK.hasPoint, BRICK.Mixed_Air_Temperature_Sensor]),
                    GraphCQ(1, [BRICK.AHU, BRICK.feeds, BRICK.Zone, BRICK.hasPoint, BRICK.Mixed_Air_Temperature_Sensor])
                    ]
             )

AHU_Tsa = CQ(description="Supply Air Temperature in the AHU",
             unit=UNIT.DEG_C,
             implementation=[
                    GraphCQ(0, [BRICK.AHU, BRICK.hasPoint, BRICK.Supply_Air_Temperature_Sensor]),
                    GraphCQ(1, [BRICK.AHU, BRICK.hasPart, BRICK.Fan, BRICK.hasPoint, BRICK.Supply_Air_Temperature_Sensor]),
                    GraphCQ(1, [BRICK.AHU, BRICK.hasPart, BRICK.Economizer, BRICK.hasPoint, BRICK.Supply_Air_Temperature_Sensor]),
                    GraphCQ(1, [BRICK.AHU, BRICK.feeds, BRICK.Zone, BRICK.hasPoint, BRICK.Supply_Air_Temperature_Sensor])
                    ]
             )

Epsilon_t = CQ(description = "Temperature threshold", 
               unit=UNIT.UNITLESS, 
               implementation=[
                   DefaultCQ(2.33)
                   ]
               ) 

DelTsf = CQ(description = "Change in temperature across supply fan", 
            unit=UNIT.DEG_C, 
            implementation=[
                DefaultCQ(1.11)
                ]
            ) 

In [6]:
# Check the shape generated
AHU_Tma.implementation[1].shape.serialize("shape1.ttl")

<Graph identifier=Na841fa4f66964436bac23c442ef7974e (<class 'rdflib.graph.Graph'>)>

# Application Authoring reusing CQs

In [3]:

### Application Authoring APAR Rule 1 ###

# pass in CQs as default arguments. The resolved function will
# have the parameters replaced with the actual resolved values
def rule1(g, sat=AHU_Tsa, mat=AHU_Tma):
    print(g.cbd(sat).serialize())
    print(f'run rule1 with {sat=}, {mat=}')
    #return mat < sat 


In [4]:
print('CQs for',rule1.__name__,'are:')
for name, cq in get_cqs(rule1).items():
    print(f"{name} => {cq}")
print('-' * 50)

CQs for rule1 are:
sat => CQ(description='Supply_Air_Temperature_in_the_AHU', unit=rdflib.term.URIRef('http://qudt.org/vocab/unit#DEG_C'), implementation=[GraphCQ(description='AHU_hasPoint_Supply_Air_Temperature_Sensor'), GraphCQ(description='AHU_hasPart_Fan_hasPoint_Supply_Air_Temperature_Sensor'), GraphCQ(description='AHU_hasPart_Economizer_hasPoint_Supply_Air_Temperature_Sensor'), GraphCQ(description='AHU_feeds_Zone_hasPoint_Supply_Air_Temperature_Sensor')])
mat => CQ(description='Mixed_Air_Temperature_in_the_AHU', unit=rdflib.term.URIRef('http://qudt.org/vocab/unit#DEG_C'), implementation=[GraphCQ(description='AHU_hasPoint_Mixed_Air_Temperature_Sensor'), GraphCQ(description='AHU_hasPart_Mixed_Damper_hasPoint_Mixed_Air_Temperature_Sensor'), GraphCQ(description='AHU_feeds_Zone_hasPoint_Mixed_Air_Temperature_Sensor')])
--------------------------------------------------


In [5]:
g = Graph()
g.parse("test_model.ttl", format='turtle')
g.parse("Brick.ttl", format='turtle')

if is_applicable(g, rule1):
    # get a list of resolved implementations
    resolved: List[Callable] = resolve(g, rule1)
    print(f"Can run the rule in {len(resolved)} place(s) in the graph")
    for fn in resolved:
        fn()
else:
    print("cannot run rule1 on the graph!")


CANNOT RUN RULE ON http://openmetrics.eu/openmetrics#AHU_1
CANNOT RUN RULE ON http://openmetrics.eu/openmetrics#AHU_4
Can run the rule in 3 place(s) in the graph
@prefix brick: <https://brickschema.org/schema/Brick#> .

<http://openmetrics.eu/openmetrics#SA_TEMP_3> a brick:Supply_Air_Temperature_Sensor .


run rule1 with sat=rdflib.term.URIRef('http://openmetrics.eu/openmetrics#SA_TEMP_3'), mat=rdflib.term.URIRef('http://openmetrics.eu/openmetrics#MA_TEMP_3')
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ns1: <https://brickschema.org/schema/Brick/ref#> .

<http://openmetrics.eu/openmetrics#SA_TEMP_2> a brick:Supply_Air_Temperature_Sensor ;
    brick:timeseries [ a ns1:TimeseriesReference ;
            ns1:hasTimeseriesId "B_01_EG_RS_EG_04_01_HVAC_VavEx_VavExAirFlRel_Present_Value" ] .


run rule1 with sat=rdflib.term.URIRef('http://openmetrics.eu/openmetrics#SA_TEMP_2'), mat=rdflib.term.URIRef('http://openmetrics.eu/openmetrics#MA_TEMP_2')
@prefix brick: <https://bri